# Putting it all together

In [1]:
import pandas as pd
from data.random_data import RandomOHLCV
# from data.ohlcv import ServeNewOHLCV
import strategies.ta as ta
import strategies.signals as sig
from frame.frame import Frame

periods = 365

ohlcv = RandomOHLCV( 
    freq      = '15 min', 
    head_max  = 0.3, 
    tail_max  = 0.3, 
    start     = '2024',           
    open_val  = 100.00,           
    periods   = periods, 
    open_rng  = (-0.4, 0.4), 
    close_rng = (-0.4, 0.4), 
    vol_rng   = (-1, 1),
    volatility_rng  = (0, 0.02),
    volatility_dur  = 3,
    volatility_freq = 50
).get_dataframe()

spy = RandomOHLCV( 
    freq      = '15 min', 
    start     = '2024',           
    open_val  = 100.00,           
    periods   = periods, 
).get_dataframe()

# ohlcv['SPY_cl'] = spy['close'] # add spy close to ohlcv




f = Frame('TSLA')
f.load_ohlcv(ohlcv)
f.import_data(spy, ['close'], prefix='SPY_')
f.add_ta(ta.ATR(span=50), {'dash': 'solid', 'color': 'cyan', 'width': 1}, row=3, chart_type='')
f.add_ta(ta.MA('close', 50), {'dash': 'solid', 'color': 'magenta', 'width': 2}, row=1, chart_type='line')
f.add_ta(ta.MA('close', 150), {'dash': 'solid', 'color': 'darkblue', 'width': 3}, row=1, chart_type='line')
f.add_ta(ta.MA('volume', 10), {'dash': 'solid', 'color': 'yellow', 'width': 1}, row=2, chart_type='line')
f.add_ta(ta.HPLP(hi_col='high', lo_col='low', span=10), 
        [{'color': 'green', 'size': 10}, # high points
          {'color': 'red', 'size': 10}], # low points
          chart_type = 'points')


# f.add_ta(ta.MACD(fast=12, slow=26, signal=9) , 
#          [{'dash': 'solid', 'color': 'purple', 'width': 2}, # MACD
#           {'dash': 'solid', 'color': 'pink', 'width': 2},   # signal
#           {'color': 'black'}], 'macd', 2) # histogram


f.add_ta(ta.SupRes(hi_point_col='HP_hi_10', lo_point_col='LP_lo_10', atr_col='ATR_50', tolerance=1),
            [{'dash': 'solid', 'color': 'green', 'fillcolour': "rgba(0, 255, 0, 0.1)", 'width': 2}, # support # green = rgba(0, 255, 0, 0.1)
            {'dash': 'solid', 'color': 'red', 'fillcolour': "rgba(255, 0, 0, 0.1)", 'width': 2}], # resistance # red = rgba(255, 0, 0, 0.1)
            chart_type = 'support_resistance') 




# f.add_ta(ta.TrendlineDetector(name='TREND', point_column='LP_lo_10',slope_direction='up', slope_tolerance=0.5, min_points=3, lookback_points=6),
#             {'dash': 'solid', 'color': 'yellow', 'width': 2}, 
#             chart_type = 'trendlines')


# f.add_ta(sig.Tail(ls='LONG', lookBack=100, normRange=(0,100), tailExceedsNthBarsAgo=2), 
#               {'dash': 'solid', 'color': 'red', 'width': 1}, 
#                chart_type = 'lines+markers', row=4) # tail signal

# f.add_ta(sig.PullbackNear(ls='LONG', lookBack=100, normRange=(0,100), maxCol='HP_hi_10', minCol='LP_lo_10',  pullbackCol='MA_cl_21', optimalRetracement=99 ),
#               {'dash': 'solid', 'color': 'blue', 'width': 1}, 
#               chart_type = 'lines+markers', row=4) # pullback signal

# f.add_ta(sig.Overlap(ls='LONG', lookBack=100, normRange=(0,100), maxCol='HP_hi_10', minCol='LP_lo_10' ),
#               {'dash': 'solid', 'color': 'purple', 'width': 1}, 
#               chart_type = 'lines+markers', row=4) # overlap signal

#! Mansfield RSI
# f.add_ta(ta.MansfieldRSI(close_col='close', market_col='SPY_cl', span=14), 
#          {'dash': 'solid', 'color': 'yellow', 'width': 2}, 
#          row=3) # mansfield rsi signal

#$ ---- Consilidation ----

f.add_ta(ta.ConsolidationZone(hp_column='HP_hi_10', lp_column='LP_lo_10', atr_column='ATR_50', price_tolerance=0.001, max_points_between=1, height_width_ratio=50, limit_zones=3),
            [{'color': "rgba(225, 182, 30, 0.5)", 'fillcolour': "rgba(225, 182, 30, 0.1)", 'width': 2}, # support # green = rgba(0, 255, 0, 0.1)
            {'color': "rgba(225, 182, 30, 0.1)", 'fillcolour': "rgba(225, 182, 30, 0.1)", 'width': 2}], # resistance # red = rgba(255, 0, 0, 0.1)
            chart_type = 'cons')

colours = ['cyan', 'yellow', 'green', 'red', 'orange', 'magenta', 'purple']


# f.add_ta(sig.ConsolidationShape(normRange=(1,25), consUpperCol='CONS_UPPER', consLowerCol='CONS_LOWER', atrCol='ATR_50', minBars=10, lookBack=periods-1),
#               [{'dash': 'solid', 'color': color, 'width': 2} for color in colours],
#               chart_type = 'line', row=4) # count

# f.add_ta(sig.ConsolidationPosition(normRange=(1,100), consUpperCol='CONS_UPPER', consLowerCol='CONS_LOWER', lookBack=periods-1),
#               [{'dash': 'solid', 'color': color, 'width': 2} for color in colours],
#               chart_type = 'line', row=4) # count

# f.add_ta(sig.ConsolidationPreMove(normRange=(1,15), consUpperCol='CONS_UPPER', consLowerCol='CONS_LOWER', maCol='MA_cl_50', atrCol='ATR_50', lookBack=periods-1),
#               [{'dash': 'solid', 'color': color, 'width': 2} for color in colours],
#               chart_type = 'line', row=4) # count

# #!! Cons Score 
# f.add_ta(sig.Score( rawName = 'Cons_Score', normRange=(0,100), lookBack=periods-1, containsAllStrings=['Cons', '1'], scoreType = 'mean', weight = 1 ),
#               {'dash': 'solid', 'color': 'brown', 'width': 4},
#               chart_type = 'line', row=4) # score signal

#$---- Trendlines ----
f.add_ta(ta.MicroTrendline(name='MTREND', pointsCol='HP_hi_10', atrCol='ATR_50', slopeDir='down', slopeToleranceATR=2, projectionPeriod=10),
            {'dash': 'solid', 'color': 'rgb(173, 90, 255)', 'width': 1}, 
            chart_type = 'trendlines')

# f.add_ta(ta.MicroTrendline(name='MTREND', pointsCol='LP_lo_10', atrCol='ATR_50', slopeDir='up', slopeToleranceATR=2, projectionPeriod=10),
#             {'dash': 'solid', 'color': 'rgb(198, 255, 90)', 'width': 1}, 
#             chart_type = 'trendlines')

# colours = ['cyan', 'yellow', 'green', 'red', 'orange', 'magenta', 'purple']

# f.add_ta(sig.CountTouches(ls='LONG', supOrRes='sup', normRange=(0,20), lookBack=periods-1, columnStartsWith='MTREND_DOWN', atrCol='ATR_50', touchTolerance=0.5),
#               [{'dash': 'solid', 'color': color, 'width': 1} for color in colours],
#               chart_type = 'line', row=4) # count touches signal

# f.add_ta(sig.LineLengths(normRange=(0,300), lookBack=periods-1, columnStartsWith='MTREND_DOWN', minBars=10),
#               [{'dash': 'solid', 'color': color, 'width': 1} for color in colours],
#               chart_type = 'line', row=4) # count touches signal


#!! Trend Score 
# f.add_ta(sig.Score( name = 'TL_DOWN_1', normRange=(0,100), lookBack=periods-1, containsAllStrings=['MTREND_DOWN', '1'], scoreType = 'mean', weight = 1 ),
#               {'dash': 'solid', 'color': colours[0], 'width': 3},
#               chart_type = 'line', row=4) # score signal

#!! Trend Score 
# f.add_ta(sig.Score( name = 'TL_DOWN_2', normRange=(0,100), lookBack=periods-1, containsAllStrings=['MTREND_DOWN', '2'], scoreType = 'mean', weight = 1 ),
#               {'dash': 'solid', 'color': colours[1], 'width': 3},
#               chart_type = 'line', row=4) # score signal

# # combined scores to allow for overlap
# f.add_ta(sig.Score( name = 'TL_DOWN', normRange=(0,100), lookBack=periods-1, containsAllStrings=['TL_DOWN'], scoreType = 'sum', weight = 1 ),
#               {'dash': 'solid', 'color': 'purple', 'width': 4},
#               chart_type = 'line', row=4) # score signal



#$ ---- Gaps----
# f.add_ta(sig.IsGappedOverPivot(ls='LONG', normRange=(0,1), pointCol='HP_hi_10',lookBack=100),
#               {'dash': 'solid', 'color': 'blue', 'width': 1},
#               chart_type = 'lines+markers', row=4) # gapped over pivot signal

# f.add_ta(sig.GappedPivots(ls='LONG', normRange=(0, 3), pointCol='HP_hi_10',  span=400, lookBack=100),
#               {'dash': 'solid', 'color': 'orange', 'width': 1}, 
#               chart_type = 'lines+markers', row=4) # gapped pivots signal

# f.add_ta(sig.GappedRetracement(ls='LONG', normRange=(0,100), pointCol='HP_hi_10', atrCol='ATR_50', lookBack=100),
#               {'dash': 'solid', 'color': 'magenta', 'width': 1},  
#               chart_type = 'lines+markers', row=4) # gapped bar quality signal


# f.add_ta(sig.GappedPastPivot(ls='LONG', normRange=(0,100), atrCol='ATR_50', pointCol='HP_hi_10', lookBack=100, maxAtrMultiple=10),
#               {'dash': 'solid', 'color': 'red', 'width': 1},
#               chart_type = 'lines+markers', row=4) # gapped past pivot signal

# f.add_ta(sig.GapSize(ls='LONG', normRange=(0,300), pointCol='HP_hi_10', atrCol='ATR_50', lookBack=100), # as pct of ATR
#               {'dash': 'solid', 'color': 'red', 'width': 1},
#               chart_type = 'lines+markers', row=4) # gap size signal
         
#!! Gaps Score 
# f.add_ta(sig.Score( name = 'L_Gaps', normRange=(0,100), lookBack=100, cols = ['SigL_GPiv', 'SigL_GRtc', 'SigL_GPP', 'SigL_GSiz'], scoreType = 'mean', weight = 1),
#               {'dash': 'solid', 'color': 'yellow', 'width': 2},
#               chart_type = 'lines+markers', row=4) # score signal

#$ ---- Volume signals ----
# f.add_ta(sig.VolumeSpike(ls='LONG', normRange=(0,200), volMACol='MA_vo_10', lookBack=100),
#               {'dash': 'solid', 'color': 'blue', 'width': 1},
#               chart_type = 'lines+markers', row=4) # volume spike signal

# f.add_ta(sig.VolumeROC(ls='LONG', normRange=(0,300), lookBack=100),
#               {'dash': 'solid', 'color': 'red', 'width': 1},
#               chart_type = 'lines+markers', row=4) # volume roc signal

# #!! Volume Score 
# f.add_ta(sig.Score( name = 'L_Vol', cols = ['SigL_VolSpike', 'SigL_VolROC'], scoreType = 'max', weight = 1, lookBack=100),
#               {'dash': 'solid', 'color': 'yellow', 'width': 2},
#               chart_type = 'lines+markers', row=4) # score signal

# #$ ---- Room To Move signals ----
# f.add_ta(sig.RoomToMove(ls='LONG', normRange=(0,10), atrCol='ATR_50', tgetCol='Res_1_Lower', lookBack=100),
#               {'dash': 'solid', 'color': 'green', 'width': 1},
#               chart_type = 'line', row=4) # room to move signal

# f.add_ta(sig.RoomToMove(ls='LONG', normRange=(0,10), atrCol='ATR_50', tgetCol='Res_2_Lower', lookBack=100),
#               {'dash': 'solid', 'color': 'cyan', 'width': 1},
#               chart_type = 'line', row=4) # room to move signal


#$ ---- Strategy signals ----
f.add_ta(sig.IsConsolidationBreakout(ls='LONG', normRange=(0,1), pointCol='LP_lo_10', valToCheck='close', consColumns=['CONS_UPPER_1', 'CONS_UPPER_2'], lookBack=periods-1),
              {'dash': 'solid', 'color': 'blue', 'width': 1},
              chart_type = 'lines+markers', row=4) # consolidation breakout signal

# bo = sig.Strategy('BO', lookBack=periods-1)
# bo.add_reset(name='Cl < PPiv', valToCheck='close', checkIf='<', colThreshold='LP_lo_10')

# bo.add_event(step=1, name='brk MA50',        valToCheck='close', checkIf='>', colThreshold='MA_cl_50')
# bo.add_event(step=1, name='brk Cons',        valToCheck=1,       checkIf='>', colThreshold='SigL_CONS_BRK')        
# bo.add_event(step=1, name='ConsSco > 60',    valToCheck=60,      checkIf='>', colThreshold='Cons_Score')    
# bo.add_event(step=1, name='VolSco > 50',     valToCheck=50,      checkIf='>', colThreshold='Score_L_Vol')
# bo.add_validation(step=1, name='Cl > MA150', valToCheck='close', checkIf='>', colThreshold='MA_cl_150')
# bo.add_validation(step=1, name='Cl > MA50',  valToCheck='close', checkIf='>', colThreshold='MA_cl_50')

# bo.add_event(step=2, name='pb Bounce',     valToCheck='HP_hi_10',  checkIf='>', colThreshold='Sup_1_Lower' )

# colours = ['cyan', 'yellow', 'green', 'red', 'orange', 'magenta', 'purple'] * 2
# f.add_ta(bo, 
#               [{'dash': 'solid', 'color': color, 'width': 2} for color in colours],
#               chart_type = 'line', row=3) # strategy

#$ ---- Utils and Ta----

# f.add_ta(sig.ExampleClass(name='Example', ls='LONG', normRange=(0,100), lookBack=100),
#               {'dash': 'solid', 'color': 'blue', 'width': 1},
#               chart_type = 'lines+markers', row=4) # example signal

# #! get HP or LP index

# # #£ get signal @ HP or LP
# f.add_ta(sig.GetSignalAtPoint(ls='LONG', normRange=(0,100), lookBack=100, pointCol='HP_hi_10', sigCol='SigL_Example', pointsAgo=1),
#               {'dash': 'solid', 'color': 'pink', 'width': 1},
#               chart_type = 'lines+markers', row=4) # example signal

# #£ get max signal since HP or LP (within nth bars after HP or LP)
# f.add_ta(sig.GetMaxSignalSincePoint(ls='LONG', normRange=(0,100), lookBack=100, pointCol='HP_hi_10', sigCol='SigL_Example', pointsAgo=1),
#               {'dash': 'solid', 'color': 'yellow', 'width': 1},
#               chart_type = 'lines+markers', row=4) # example signal



#$ ---- Pullback Bounce signals ----

#£ recent hp low > current support
#£ since recent HP lowest point < current support
#£ close > support
# f.add_ta(sig.IsPullbackBounce(ls='LONG', lookBack=100, pointCol='HP_hi_10', supResCol='Sup_1_Lower'),
#               {'dash': 'solid', 'color': 'blue', 'width': 1},
#               chart_type = 'lines+markers', row=4) # pullback bounce signal

#£ Volume spike on recnt HP > 80% of the MA10 volume

#£ retracment ideally 50% of the recent HP
# f.add_ta(sig.Trace(ls='LONG', normRange=(0,100), pointCol='HP_hi_10', atrCol='ATR_50', lookBack=100),

#£ lower highs on the retracement

#£ overlap on the retracement

#£ Retraces to the MA21
#£ Retraces to the MA50

#£ recent low point has long tail


#$ ---- BreakOut signals ----

#£ Crossed above resistance (bar low > resistance, prev bar lowerer than resistance)

#£ Volume spike on breakout > 80% of the MA10 volume

#! Resistance is significant (number of touches)

#! breaks consolidation


#$ ---- Prev Resistance Significance signals (now Support) ----

#! number of touches




f.setup_chart() 
f.plot(trading_hours=True, height=1000, width=1500)


"""
Resistance: 


The objective is to modify a support and resistance class that will find the support and resistance levels and return a pandas data frame with those levels projected forward to the end of the data frame from the points that have been detected
So it is important that every row of the data frame is filled in with the same price point stretching from the index where the point is located to the end of the data frame so that it displays correctly on a chart. (This is not a visualisation task but a data manipulation task)
Do not worry about tolerances and touch points for now.

  -- All code snippets are to be returned in the chat do not update files directly
  -- compute only base on the last close price on the last bar
  -- hp and lp is high point or low point 
  -- HP and LP are already computed and should be obtained from the data frame by referencing the relevant columns
  -- find the next  hp or lp That is higher than the close price. I suggest putting them all in a list both high points and low points and sorting them in ascending order first.
  -- When finding the next higher point use the argument pointsAgo to find the point that is x points higher from the current point.
  -- now find to index Of the point that was found whether it was a low point or a high point is irrelevant. 
  -- Now project the point that was found and forward fill it to the end of the df So that when visualised on a chart there will be a line stretching from the point that has been found to the end of the chart. 
  
  Support:
  -- Similar to resistance but of course the opposite.

  """

# name = f"{self.name}_{self.slopeDir.upper()}_{i+1}"
cols = [c for c in f.data.columns if 'Stgy' in c]
f.data[cols].dropna(how='all')



""
date


In [3]:
spy['close'].iat[-1] = 3000
spy['open'].iat[-1] = 6000

f.import_data(spy, ['close', 'open'], prefix='SPY_')
f.data.columns


Index(['open', 'high', 'low', 'close', 'volume', 'ATR_50', 'MA_cl_50',
       'MA_cl_150', 'MA_vo_10', 'HP_hi_10', 'LP_lo_10', 'Res_1', 'Res_1_Upper',
       'Res_1_Lower', 'Res_2', 'Res_2_Upper', 'Res_2_Lower', 'Sup_1',
       'Sup_1_Upper', 'Sup_1_Lower', 'Sup_2', 'Sup_2_Upper', 'Sup_2_Lower',
       'CONS_UPPER_1', 'CONS_LOWER_1', 'CONS_UPPER_2', 'CONS_LOWER_2',
       'MTREND_DOWN_1', 'MTREND_DOWN_2', 'MTREND_DOWN_3',
       'SigL_CONS_BRK_close', 'SPY_close', 'SPY_open'],
      dtype='object')

In [10]:
f = Frame('TSLA', run_ta_on_load=True)
f.load_ohlcv(ohlcv)

# Let's add the TAs one by one and check the data after each addition
f.add_ta(ta.ATR(span=50), {'dash': 'solid', 'color': 'cyan', 'width': 1}, row=3, chart_type='line')  # Added chart_type='line'

# Check if ATR column exists
print("Columns after ATR:", f.data.columns.tolist())

f.add_ta(ta.MA('close', 50), {'dash': 'solid', 'color': 'magenta', 'width': 2}, row=1, chart_type='line')  # Added chart_type='line'

# Check if MA50 column exists
print("Columns after MA50:", f.data.columns.tolist())

f.add_ta(ta.MA('close', 150), {'dash': 'solid', 'color': 'darkblue', 'width': 3}, row=1, chart_type='line')  # Added chart_type='line'
f.add_ta(ta.MA('volume', 10), {'dash': 'solid', 'color': 'yellow', 'width': 1}, row=2, chart_type='line')  # Added chart_type='line'

f.add_ta(ta.HPLP(hi_col='high', lo_col='low', span=10), 
        [{'color': 'green', 'size': 10}, # high points
          {'color': 'red', 'size': 10}], # low points
          chart_type='points')

# Check final columns
print("Final columns:", f.data.columns.tolist())

# Plot and check if the lines appear now
f.plot(use_backtest_data=False)

Columns after ATR: ['open', 'high', 'low', 'close', 'volume', 'SPY_cl', 'ATR_50']
Columns after MA50: ['open', 'high', 'low', 'close', 'volume', 'SPY_cl', 'ATR_50', 'MA_cl_50']
Final columns: ['open', 'high', 'low', 'close', 'volume', 'SPY_cl', 'ATR_50', 'MA_cl_50', 'MA_cl_150', 'MA_vo_10', 'HP_hi_10', 'LP_lo_10']


In [11]:
# Check the first few rows of MA columns
print("MA 50 values:")
print(f.data['MA_cl_50'].head())
print("\nMA 150 values:")
print(f.data['MA_cl_150'].head())


MA 50 values:
date
2024-01-01 00:00:00   NaN
2024-01-01 00:15:00   NaN
2024-01-01 00:30:00   NaN
2024-01-01 00:45:00   NaN
2024-01-01 01:00:00   NaN
Name: MA_cl_50, dtype: float64

MA 150 values:
date
2024-01-01 00:00:00   NaN
2024-01-01 00:15:00   NaN
2024-01-01 00:30:00   NaN
2024-01-01 00:45:00   NaN
2024-01-01 01:00:00   NaN
Name: MA_cl_150, dtype: float64


In [1]:
import pandas as pd
from data.random_data import RandomOHLCV
# from data.ohlcv import ServeNewOHLCV
import strategies.ta as ta
import strategies.signals as sig
from frame.frame import Frame

periods = 365

ohlcv = RandomOHLCV( 
    freq      = '15 min', 
    head_max  = 0.3, 
    tail_max  = 0.3, 
    start     = '2024',           
    open_val  = 100.00,           
    periods   = periods, 
    open_rng  = (-0.4, 0.4), 
    close_rng = (-0.4, 0.4), 
    vol_rng   = (-1, 1),
    volatility_rng  = (0, 0.02),
    volatility_dur  = 3,
    volatility_freq = 50
).get_dataframe()

spy = RandomOHLCV( 
    freq      = '15 min', 
    start     = '2024',           
    open_val  = 100.00,           
    periods   = periods, 
).get_dataframe()
f = Frame('TSLA')
f.load_ohlcv(ohlcv)

# Add indicators
f.add_ta(ta.MA('close', 50), {'dash': 'solid', 'color': 'magenta', 'width': 2}, row=1, chart_type='line')
f.add_ta(ta.MA('close', 150), {'dash': 'solid', 'color': 'darkblue', 'width': 3}, row=1, chart_type='line')
f.add_ta(ta.ATR(span=50), {'dash': 'solid', 'color': 'cyan', 'width': 1}, row=3, chart_type='line')
f.add_ta(ta.MA('volume', 10), {'dash': 'solid', 'color': 'yellow', 'width': 1}, row=2, chart_type='line')
f.add_ta(ta.HPLP(hi_col='high', lo_col='low', span=10), 
        [{'color': 'green', 'size': 10}, 
         {'color': 'red', 'size': 10}], 
        chart_type='points')

# Add debug prints
print("\nStored TA indicators:")
for ta_obj, style, chart_type, row in f.ta:
    print(f"Indicator: {ta_obj.names}, Style: {style}, Chart Type: {chart_type}, Row: {row}")
    print(f"Indicator columns: {ta_obj.names}")

# Plot
f.plot()


Stored TA indicators:
Indicator: MA_cl_50, Style: {'dash': 'solid', 'color': 'magenta', 'width': 2}, Chart Type: line, Row: 1
Indicator columns: MA_cl_50
Indicator: MA_cl_150, Style: {'dash': 'solid', 'color': 'darkblue', 'width': 3}, Chart Type: line, Row: 1
Indicator columns: MA_cl_150
Indicator: ['ATR_50'], Style: {'dash': 'solid', 'color': 'cyan', 'width': 1}, Chart Type: line, Row: 3
Indicator columns: ['ATR_50']
Indicator: MA_vo_10, Style: {'dash': 'solid', 'color': 'yellow', 'width': 1}, Chart Type: line, Row: 2
Indicator columns: MA_vo_10
Indicator: ['HP_hi_10', 'LP_lo_10'], Style: [{'color': 'green', 'size': 10}, {'color': 'red', 'size': 10}], Chart Type: points, Row: 1
Indicator columns: ['HP_hi_10', 'LP_lo_10']

Processing indicator:
Names to check: ['MA_cl_50']
Available columns: ['open', 'high', 'low', 'close', 'volume', 'MA_cl_50', 'MA_cl_150', 'ATR_50', 'MA_vo_10', 'HP_hi_10', 'LP_lo_10']
Found columns: ['MA_cl_50']
Data shape for ['MA_cl_50']: (365, 1)
First few values

In [14]:
print("Data columns before plotting:", f.data.columns.tolist())

Data columns before plotting: ['open', 'high', 'low', 'close', 'volume', 'SPY_cl', 'MA_cl_50', 'MA_cl_150', 'ATR_50', 'MA_vo_10', 'HP_hi_10', 'LP_lo_10']


In [21]:
def find_object_by_name(object_list, target_name, namesExtend=None):
    """
    Iterates through a list of tuples containing objects and returns the names
    of the object that matches the target_name.
    
    Args:
        object_list: List of tuples where first element is an object
        target_name: Name to search for
        
    Returns:
        Names of the matching object or None if not found
    """
    for obj_tuple in object_list:
        obj = obj_tuple[0]  # Get the object from the tuple
        if hasattr(obj, 'name') and obj.name == target_name:
            if namesExtend:
                return [n for n in obj.names if namesExtend in n]
            return obj.names
    return None

# Example usage:
names = find_object_by_name(f.ta, "MTREND", namesExtend='UP')
# names = find_object_by_name(f.ta, "MTREND", namesExtend=None)
names

[]

In [1]:

    """
    Here I have a class outline. Please complete the class. look at the dock string and comments to understand the intent of the class. 
    You must maintain the _compute_row method as this class inherits from the Signals class which uses a run method which calls the _compute_row  method for each row.
    
    The columns to be computed from will be defined by searching using lineColStartsWith variable. 
    Each column that starts with this variable will also have a number appended to it. eg MTREND_DOWN_1, MTREND_DOWN_2, ...
    The computed output will be a corresponding column with the same name as the input column but with the word touches appended to it.
    The output will be computed on from the perspective of the last row of the data frame. (the _compute_row method is called for each row of the data frame)
    _compute_row should retrun a series with a vlaue for each column that is being computed. Note that it is just return a row of the data frame.




'Here I have a class outline. Please complete the class look at the dock string hopefully it is clear enough. \nYou must maintain the compute row method as this class inherits from signals which uses a run method which calls this compute method for each row.\nFor this particular class it needs to count the number of touches of a line which can be a resistance or support line it can be a trend line or it can be the upper or lower bounds of consolidation zones\nThe the function of the line is not necessarily important but now that some lines will be horizontal meaning that they are maintaining the same price level and other lines such as trend lines will be on a slope which means every single bar has a different value\n\nTo add further complexity each line line has its own separate column within the pandas data frame. So even though I have multiple say trend lines or consolidation zones within my data frame they all have their own columns for each line \n  The way to Identify the columns

In [4]:

def daily_breakout(f):
    #£ gap score
    #£ has room to move
    #£ volume spike
    #£ prev period was consolidation
    #£ period prior to consolidation was a pullback
    #£ is a bounce from breakout
    #£ MArket Relative strength
    #£ Sector Relative strength

def buy_set_up(f):
    # 1) Must be in a stage 2 uptrend or coming from a double bottom retest/transition. (W pattern)
    # 2) 2 or more lower highs (LH)
    # 3) ‘Sequential’ pullback with less than a 50% overlap on any bar. (A 45° (degree) angle of retracement is ideal. Don’t want it ‘too’ steep.)

SyntaxError: incomplete input (1224903109.py, line 8)

In [ ]:
@dataclass
class Strategy(Signals):
    name: str = ''
    ls:str = 'LONG'

    def __post_init__(self):
        self.name = f'Stgy_{self.name}'
        self.names = [self.name]
        self.event_tracker = {}

    def add_event(self, step, name, valToCheck, checkIf, colThreshold ):
        """An event happend just once and triggers a validation. is not reversible"""
        self.event_tracker[name] = {
            'step': step, 
            'type': 'event', 
            'status': 'pending',
            'valToCheck': valToCheck,
            'checkIf': checkIf,
            'colThreshold': colThreshold,
            'isTrue': 0
        }
        self.names += [f'{self.name}_{name}']

    def add_validation(self, step, name, valToCheck, checkIf, colThreshold):
        """A Validation is an ongoing check that is reversible.
        """
        self.event_tracker[name] = {
            'step': step, 
            'type': 'validation', 
            'status': 'pending',
            'valToCheck': valToCheck,
            'checkIf': checkIf,
            'colThreshold': colThreshold,
            'isTrue': 0
        }
        self.names += [f'{self.name}_{name}']

    def add_reset(self, name, valToCheck, checkIf, colThreshold):
        """If resetIfInvalid is set to True and the Validation fails the entire strategy is is canceled and marked as invalid"""
        self.event_tracker[name] = {
            'type': 'reset',
            'status': 'pending',
            'valToCheck': valToCheck,
            'checkIf': checkIf,
            'colThreshold': colThreshold,
            'isTrue': 0
        }
        self.names += [f'{self.name}_{name}']

    def is_reset_triggered(self):
        # check if the reset event has been triggered
        # return True or False
        pass

    def set_triggered_events(self):
        # loops through the events sets the isTrue value to 1 if the condition is met
        # if isTrue is already 1 then skip
        # only check next step once all events in the current step are valid
        pass

    def set_triggered_validations(self):
        # loops through the validations and sets the isTrue value to 1 if the condition is met and 0 if not
        # only check next step once all events in the current step are valid
        pass

    def update_status(self):
        # checks if all the events and validations are valid
        # if all are valid then set the status to 'valid'
        pass

    def reset(self):
        # resets the strategy to its initial state
        pass

    def _compute_row(self, df):
        
        # 1. check if the strategy should be cancelled
        if self.is_reset_triggered():
            self.reset()
            return 0
        
        # 2. loop through the events and validations
        self.set_triggered_events()
        self.set_triggered_validations()

        # 3. update the status of the strategy and assign values to results

        # Initialize results DataFrame
        results = pd.DataFrame(index=df.index, columns=self.names)
        for k, v in self.event_tracker.items():
            results[f'{self.name}_{k}'] = v['isTrue']

        step1_pct_complete = sum([v['isTrue'] for k, v in self.event_tracker.items() if v['step'] == 1]) / len([v for k, v in self.event_tracker.items() if v['step'] == 1])
        step2_pct_complete = sum([v['isTrue'] for k, v in self.event_tracker.items() if v['step'] == 2]) / len([v for k, v in self.event_tracker.items() if v['step'] == 2])
        total_pct_complete = (step1_pct_complete + step2_pct_complete) / 2

        results[f'{self.name}_step1'] = step1_pct_complete
        results[f'{self.name}_step2'] = step2_pct_complete
        results[f'{self.name}_total'] = total_pct_complete

        return results




bo = Strategy('BO_Daily')
bo.add_reset(name='Cl < PPiv', valToCheck='close', checkIf='<', colThreshold='LP_lo_10')

bo.add_event(step=1, name='brk MA50',        valToCheck='close', checkIf='>', colThreshold='MA_cl_50')
bo.add_event(step=1, name='brk Cons',        valToCheck='close', checkIf='>', colThreshold='CONS_UPPER_1')        
bo.add_event(step=1, name='ConsSco > 60',    valToCheck=60,      checkIf='>', colThreshold='Cons_Score')    
bo.add_event(step=1, name='VolSco > 50',     valToCheck=50,      checkIf='>', colThreshold='Score_L_Vol')
bo.add_validation(step=1, name='Cl > MA150', valToCheck='close', checkIf='>', colThreshold='MA_cl_150')
bo.add_validation(step=1, name='Cl > MA50',  valToCheck='close', checkIf='>', colThreshold='MA_cl_50')

bo.add_event(step=2, name='pb Bounce',     valToCheck='HP_hi_10',  checkIf='>', colThreshold='Sup_1_Lower' )


bo.run(df)

In [ ]:
col1 = 'BO_brk MA50' # value 100 if the condition is met
col2 = 'BO_brk Cons' # value 100 if the condition is met
col3 = 'BO_ConsSco > 60' # value 100 if the condition is met
col4 = 'BO_VolSco > 50' # value 100 if the condition is met
col5 = 'BO_Cl > MA150' # value 100 if the condition is met
col6 = 'BO_Cl > MA50' # value 100 if the condition is met
col7 = 'BO_pb Bounce' # value 100 if the condition is met
col8 = 'BO_step1' # pct complete of step 1
col9 = 'BO_step2' # pct complete of step 2
col10 = 'BO_total' # total pct complete

# Backtester

In [32]:
from frame.frame import Frame
from dataclasses import dataclass, field
from data import historical_data as hd
from ib_insync import *
from typing import Dict, Any, List
import pandas as pd
from data.random_data import RandomOHLCV
# from data.ohlcv import ServeNewOHLCV
import strategies.ta as ta
import strategies.signals as sig

@dataclass
class TAData:
    ta: ta.TA
    style: Dict[str, Any] | List[Dict[str, Any]] = field(default_factory=dict)
    chart_type: str = "line"
    row: int = 1



@dataclass
class StockX:
    ib: IB = None
    symbol: str = ''

    def __post_init__(self):
        self.fundamentals = {}
        self.frames = {}
    
    def set_up_frame(self, timeframe, dataType:str='random', start_date:str="52 weeksAgo", end_date:str='now'):
        self.frames[timeframe] = Frame(self.symbol, run_ta_on_load=True)
        
        if dataType == 'random':
            df =  RandomOHLCV( 
            freq      = timeframe, 
            head_max  = 0.3, 
            tail_max  = 0.3, 
            start     = '2024',           
            open_val  = 100.00,           
            periods   = 400, 
            open_rng  = (-0.4, 0.4), 
            close_rng = (-0.4, 0.4), 
            vol_rng   = (-1, 1),
            volatility_rng  = (0, 0.02),
            volatility_dur  = 3,
            volatility_freq = 50).get_dataframe()

            self.frames[timeframe].load_ohlcv(df)

        elif dataType == 'ohlcv':
             self.frames[timeframe].load_ohlcv(hd.get_hist_data(self.symbol, start_date, end_date, timeframe))

        elif dataType == 'tick':
            # todo: implement tick data
            pass


    def req_fundamentals(self):
        # request the fundamentals from the data source
        pass

    def req_ohlcv(self, timeframe):
        # request the ohlcv from the data source
        pass

    def req_tick_data(self, timeframe):
        # request the tick data from the data source
        pass

    def add_ohlcv(self, timeframe, ohlcv):
        # add the ohlcv to the data frame 
        # can be used to add market data or other data not just the open high low close
        pass

    def add_rows(self, timeframe, rows):
        # add the rows to the data frame.  eg if new data such as market data is added
        pass

    def add_ta(self, timeframe, taList:list[TAData]):
        # add the ta to the data frame
        for ta in taList:
            self.frames[timeframe].add_ta(ta.ta, ta.style, ta.chart_type, ta.row)
        pass

    def run_backtest_old(self, start: str | int, end: str | int, save_snapshots: bool = False):
        """
        Run backtest across multiple timeframes.
        
        Args:
            start: Start datetime or index (can be negative for indexing from end)
            end: End datetime or index (can be negative for indexing from end)
            save_snapshots: Whether to save snapshots during backtesting
        """
        print(f"\nInitializing backtest with parameters:")
        print(f"Start: {start}, End: {end}")
        
        # Get the smallest timeframe 
        smallest_tf_frame = None
        smallest_frequency = pd.Timedelta.max
        
        for frame_name, frame in self.frames.items():
            if not frame.data.empty:
                frequencies = pd.Series(frame.data.index[1:] - frame.data.index[:-1]).mode()
                if not frequencies.empty:
                    current_frequency = frequencies.iloc[0]
                    if current_frequency < smallest_frequency:
                        smallest_frequency = current_frequency
                        smallest_tf_frame = frame
                        smallest_tf_name = frame_name
        
        if smallest_tf_frame is None:
            raise ValueError("No valid timeframes found")
        
        # Initialize backtest for smallest timeframe first
        smallest_tf_frame.backtest_setup(start, end, save_snapshots)
        start_time = smallest_tf_frame.data.index[smallest_tf_frame._backtest_start_idx]
        end_time = smallest_tf_frame.data.index[smallest_tf_frame._backtest_end_idx]
        
        # Initialize other timeframes using the same datetime range
        for frame_name, frame in self.frames.items():
            if frame_name != smallest_tf_name:
                # Find the indices in this timeframe that match our datetime range
                start_idx = frame.data.index.get_indexer([start_time])[0]
                end_idx = frame.data.index.get_indexer([end_time])[0]
                frame.backtest_setup(start_idx, end_idx, save_snapshots)
        
        # Run the backtest using the smallest timeframe as the driver
        running = True
        while running:
            # Update smallest timeframe
            if not smallest_tf_frame.backtest_next_row():
                running = False
                continue
                
            current_time = smallest_tf_frame.backtest_data.index[-1]
            
            # Only update other timeframes when it's time for their new bar
            for frame_name, frame in self.frames.items():
                if frame_name == smallest_tf_name:
                    continue
                    
                if current_time > frame.backtest_data.index[-1]:
                    frame.backtest_next_row()
            
    def run_backtest(self, start: str | int, end: str | int, htf_imports: Dict[str, list] = None, save_snapshots: bool = False):
        """
        Run backtest with higher timeframe data importing.
        
        Args:
            start: Start datetime or index
            end: End datetime or index
            htf_imports: Dict mapping timeframe to columns to import:
                {'4H': ['close', 'volume']}
            save_snapshots: Whether to save snapshots
        """
        print(f"\nInitializing backtest with parameters:")
        print(f"Start: {start}, End: {end}")
        
        smallest_tf_frame = None
        smallest_frequency = pd.Timedelta.max
        
        for frame_name, frame in self.frames.items():
            if not frame.data.empty:
                frequencies = pd.Series(frame.data.index[1:] - frame.data.index[:-1]).mode()
                if not frequencies.empty:
                    current_frequency = frequencies.iloc[0]
                    if current_frequency < smallest_frequency:
                        smallest_frequency = current_frequency
                        smallest_tf_frame = frame
                        smallest_tf_name = frame_name
        
        if smallest_tf_frame is None:
            raise ValueError("No valid timeframes found")
        
        # Initialize backtests
        smallest_tf_frame.backtest_setup(start, end, save_snapshots)
        start_time = smallest_tf_frame.data.index[smallest_tf_frame._backtest_start_idx]
        end_time = smallest_tf_frame.data.index[smallest_tf_frame._backtest_end_idx]
        
        for frame_name, frame in self.frames.items():
            if frame_name != smallest_tf_name:
                start_idx = frame.data.index.get_indexer([start_time])[0]
                end_idx = frame.data.index.get_indexer([end_time])[0]
                frame.backtest_setup(start_idx, end_idx, save_snapshots)
        
        running = True
        while running:
            # Import HTF data before running next row
            if htf_imports:
                for htf_name, columns in htf_imports.items():
                    if htf_name in self.frames:
                        htf_frame = self.frames[htf_name]
                        smallest_tf_frame.import_data(
                            htf_frame.backtest_data,
                            columns,
                            merge_to_backtest=True
                        )
            
            # Update smallest timeframe with imported data
            if not smallest_tf_frame.backtest_next_row():
                running = False
                continue
                
            current_time = smallest_tf_frame.backtest_data.index[-1]
            
            # Update other timeframes when needed
            for frame_name, frame in self.frames.items():
                if frame_name == smallest_tf_name:
                    continue
                    
                if current_time > frame.backtest_data.index[-1]:
                    # Import HTF data before running next row for intermediate timeframes
                    if htf_imports:
                        for htf_name, columns in htf_imports.items():
                            if htf_name in self.frames:
                                htf_frame = self.frames[htf_name]
                                frame.import_data(
                                    htf_frame.backtest_data,
                                    columns,
                                    merge_to_backtest=True
                                )
                                
                    frame.backtest_next_row()



ib = IB()

sx = StockX(ib,'TSLA')
sx.set_up_frame('1 min')
sx.set_up_frame('2 mins')
sx.set_up_frame('5 mins')
# sx.set_up_frame('15 mins')

sx.frames['1 min'].import_data(sx.frames['5 mins'].data, ['close'], prefix='5min_')




ta_list = [
    TAData(ta.ATR(span=50), {'dash': 'solid', 'color': 'cyan', 'width': 1}, row=3, chart_type=''),
    TAData(ta.MA('close', 50), {'dash': 'solid', 'color': 'magenta', 'width': 2}),
    TAData(ta.MA('close', 150), {'dash': 'solid', 'color': 'darkblue', 'width': 3}),
    TAData(ta.MA('volume', 10), {'dash': 'solid', 'color': 'yellow', 'width': 1}, row=2),
    TAData(ta.HPLP(hi_col='high', lo_col='low', span=10), 
        [{'color': 'green', 'size': 10}, # high points
        {'color': 'red', 'size': 10}], # low points
        chart_type = 'points')
]

ta_1min =[TAData(ta.MA('5min_close', 10), {'dash': 'solid', 'color': 'white', 'width': 2})]

sx.add_ta('1 min', ta_list + ta_1min)
sx.add_ta('2 mins', ta_list)
sx.add_ta('5 mins', ta_list )


htf_imports = {
    '2 mins': ['close', 'volume'],
    '5 mins': ['close']
}

sx.run_backtest(-50, -1, save_snapshots=True, htf_imports=htf_imports)




Initializing backtest with parameters:
Start: -50, End: -1


In [30]:
sx.frames['1 min'].data

,open,high,low,close,volume,5min_close,ATR_50,MA_cl_50,MA_cl_150,MA_vo_10,HP_hi_10,LP_lo_10
date,,,,,,,,,,,,
2024-01-01 00:00:00,99.86,100.12,99.55,99.59,100,99.93,0.570000,NaN,NaN,NaN,NaN,NaN
2024-01-01 00:01:00,99.55,99.97,99.35,99.75,100,99.93,0.595000,NaN,NaN,NaN,NaN,NaN
2024-01-01 00:02:00,100.01,100.14,99.72,100.02,100,99.93,0.536667,NaN,NaN,NaN,100.14,NaN
2024-01-01 00:03:00,99.72,99.79,99.40,99.53,100,99.93,0.500000,NaN,NaN,NaN,NaN,NaN
2024-01-01 00:04:00,99.32,99.44,99.23,99.40,64,99.93,0.442000,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-01 06:35:00,93.15,93.36,92.90,93.17,5,106.99,0.464400,91.6248,91.423400,8.7,NaN,NaN
2024-01-01 06:36:00,93.25,93.32,92.87,92.95,5,106.99,0.462400,91.6784,91.425933,8.3,NaN,NaN
2024-01-01 06:37:00,93.09,93.58,92.90,93.37,9,106.99,0.465800,91.7306,91.429800,8.7,NaN,NaN


In [36]:
f = sx.frames['1 min']
# f.setup_chart() 
f.backtest_data
sx.frames['5 mins'].get_snapshot(0, plot=True, width=1000, height=600)
sx.frames['1 min'].get_snapshot(0, plot=True, width=1000, height=600)

Viewing snapshot 1/50 - Date: 2024-01-01 05:50:00 (location 350)


,open,high,low,close,volume,ATR_50,MA_cl_50,MA_cl_150,MA_vo_10,HP_hi_10,LP_lo_10
date,,,,,,,,,,,
2024-01-01 00:00:00,100.33,100.35,100.11,100.24,100,0.2400,NaN,NaN,NaN,NaN,NaN
2024-01-01 00:01:00,100.32,100.57,100.22,100.51,100,0.2950,NaN,NaN,NaN,NaN,NaN
2024-01-01 00:02:00,100.60,100.89,100.04,100.26,37,0.4800,NaN,NaN,NaN,100.89,NaN
2024-01-01 00:03:00,100.63,100.82,100.36,100.40,50,0.4750,NaN,NaN,NaN,NaN,NaN
2024-01-01 00:04:00,100.69,100.77,100.49,100.72,48,0.4360,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2024-01-01 05:45:00,99.43,99.58,99.23,99.31,55,0.4954,99.0458,98.279867,56.5,NaN,NaN
2024-01-01 05:46:00,99.11,99.61,99.10,99.35,39,0.4922,99.0448,98.303867,54.9,NaN,NaN
2024-01-01 05:47:00,99.40,99.50,99.10,99.34,15,0.4884,99.0496,98.326400,51.7,NaN,NaN


In [14]:
sx.frames['2 mins'].get_snapshot(10, plot=True, width=1000, height=600)

Viewing snapshot 11/26 - Date: 2024-01-01 06:08:00 (location 184)


,open,high,low,close,volume,ATR_50,MA_cl_50,MA_cl_150,MA_vo_10,HP_hi_10,LP_lo_10,htf_volume,htf_close
date,,,,,,,,,,,,,
2024-01-01 00:00:00,99.62,100.25,99.56,99.97,100,0.690000,NaN,NaN,NaN,NaN,NaN,100.0,99.64
2024-01-01 00:02:00,100.36,100.48,100.10,100.12,100,0.535000,NaN,NaN,NaN,NaN,NaN,100.0,99.64
2024-01-01 00:04:00,100.47,100.75,100.13,100.17,54,0.563333,NaN,NaN,NaN,NaN,NaN,54.0,99.64
2024-01-01 00:06:00,100.56,100.59,100.01,100.28,32,0.567500,NaN,NaN,NaN,NaN,NaN,32.0,99.62
2024-01-01 00:08:00,100.68,100.92,100.16,100.37,41,0.606000,NaN,NaN,NaN,NaN,NaN,41.0,99.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-01 06:00:00,96.17,96.42,95.99,96.32,64,0.446000,95.0662,NaN,30.7,NaN,NaN,64.0,101.24
2024-01-01 06:02:00,96.29,96.68,96.04,96.45,95,0.448200,95.0968,NaN,35.4,NaN,NaN,95.0,101.24
2024-01-01 06:04:00,96.29,96.57,95.93,95.96,100,0.449200,95.1168,NaN,43.6,NaN,NaN,100.0,101.24


In [33]:
sx.frames['1 min'].run_snapshots(0,50, plot=True, display_df=False, sleep_time=0.1, width=2000, height=1500)
# sx.frames['5 mins'].run_snapshots(0,50, plot=True, display_df=False, sleep_time=0.5, width=1000, height=600)

Viewing snapshot 50/50 - Date: 2024-01-01 06:38:00 (location 398)


In [8]:
import pandas as pd
import numpy as np
import pickle
import gzip
import os

def analyze_storage_requirements():
    # Create sample DataFrame (50 columns, 1000 rows)
    df = pd.DataFrame({
        'timestamp': pd.date_range('2024-01-01', periods=1000, freq='1min'),
        'price': np.random.random(1000) * 100,
        'volume': np.random.randint(1000, 100000, 1000)
    })
    
    # Add technical analysis columns
    for i in range(47):  # Adding 47 more to reach 50 columns
        df[f'indicator_{i}'] = np.random.random(1000)
    
    # Create a temporary directory for our test
    os.makedirs('temp_storage', exist_ok=True)
    
    # Test different storage methods by actually writing files
    
    # Regular CSV
    csv_path = 'temp_storage/test.csv'
    df.to_csv(csv_path)
    csv_size = os.path.getsize(csv_path)
    
    # Regular Pickle
    pickle_path = 'temp_storage/test.pkl'
    df.to_pickle(pickle_path)
    pickle_size = os.path.getsize(pickle_path)
    
    # Compressed CSV
    csv_compressed_path = 'temp_storage/test.csv.gz'
    df.to_csv(csv_compressed_path, compression='gzip')
    csv_compressed_size = os.path.getsize(csv_compressed_path)
    
    # Compressed Pickle
    pickle_compressed_path = 'temp_storage/test.pkl.gz'
    with gzip.open(pickle_compressed_path, 'wb') as f:
        pickle.dump(df, f)
    pickle_compressed_size = os.path.getsize(pickle_compressed_path)
    
    # Print results
    print(f"Single DataFrame (1000 rows × 50 columns):")
    print(f"Regular CSV: {csv_size / (1024 * 1024):.2f} MB")
    print(f"Regular Pickle: {pickle_size / (1024 * 1024):.2f} MB")
    print(f"Compressed CSV: {csv_compressed_size / (1024 * 1024):.2f} MB")
    print(f"Compressed Pickle: {pickle_compressed_size / (1024 * 1024):.2f} MB")
    
    # Calculate for 4 frames per stock
    frames_per_stock = 4
    total_csv = csv_size * frames_per_stock
    total_pickle = pickle_size * frames_per_stock
    total_compressed_csv = csv_compressed_size * frames_per_stock
    total_compressed_pickle = pickle_compressed_size * frames_per_stock
    
    print(f"\nOne stock (4 frames):")
    print(f"Total CSV: {total_csv / (1024 * 1024):.2f} MB")
    print(f"Total Pickle: {total_pickle / (1024 * 1024):.2f} MB")
    print(f"Total Compressed CSV: {total_compressed_csv / (1024 * 1024):.2f} MB")
    print(f"Total Compressed Pickle: {total_compressed_pickle / (1024 * 1024):.2f} MB")
    
    # Calculate for full trading day (6.5 hours of 1-minute data)
    trading_minutes = int(6.5 * 60)  # 390 minutes
    daily_csv = total_csv * trading_minutes
    daily_pickle = total_pickle * trading_minutes
    daily_compressed_csv = total_compressed_csv * trading_minutes
    daily_compressed_pickle = total_compressed_pickle * trading_minutes
    
    print(f"\nOne stock full trading day (390 minutes):")
    print(f"Daily CSV: {daily_csv / (1024 * 1024 * 1024):.2f} GB")
    print(f"Daily Pickle: {daily_pickle / (1024 * 1024 * 1024):.2f} GB")
    print(f"Daily Compressed CSV: {daily_compressed_csv / (1024 * 1024 * 1024):.2f} GB")
    print(f"Daily Compressed Pickle: {daily_compressed_pickle / (1024 * 1024 * 1024):.2f} GB")
    
    # Cleanup
    for file in ['test.csv', 'test.pkl', 'test.csv.gz', 'test.pkl.gz']:
        try:
            os.remove(os.path.join('temp_storage', file))
        except:
            pass
    try:
        os.rmdir('temp_storage')
    except:
        pass

# Run the analysis
analyze_storage_requirements()



Single DataFrame (1000 rows × 50 columns):
Regular CSV: 0.91 MB
Regular Pickle: 0.38 MB
Compressed CSV: 0.41 MB
Compressed Pickle: 0.35 MB

One stock (4 frames):
Total CSV: 3.64 MB
Total Pickle: 1.53 MB
Total Compressed CSV: 1.66 MB
Total Compressed Pickle: 1.42 MB

One stock full trading day (390 minutes):
Daily CSV: 1.39 GB
Daily Pickle: 0.58 GB
Daily Compressed CSV: 0.63 GB
Daily Compressed Pickle: 0.54 GB
